# Сравнение fft с дуболомным преобразованием Фурье

Из numpy нам потребуется только функции для пи и для экспоненты. Чтобы любой смог увидеть, что его часть с fft трогать не будем, импортируем только указанные функции

In [1]:
from numpy import pi as nppi
from numpy import exp as npexp

Введём функцию дуболомных прямого и обратного преобразований Фурье, как завещал Numerical recipes

In [2]:
def dubolom(x):
    n = len(x)
    result = [None] * n
    W = [npexp(-2j * nppi * i / n) for i in range(n)]
    for j in range(n):
        result[j] = sum([W[i] ** j * x[i] for i in range(n)])
    return result   

def idubolom(x):
    n = len(x)
    result = [None] * n
    W = [npexp(2j * nppi * i / n) for i in range(n)]
    for j in range(n):
        result[j] = sum([W[i] ** j * x[i] / n for i in range(n)])
    return result   

То же для ifft

In [3]:
def fft(x):
    n = len(x)
    if n == 1: 
        return x
    chet = fft(x[0::2])
    nechet =  fft(x[1::2])
    T = [npexp(-2j * nppi * i / n) * nechet[i] for i in range(n // 2)]
    result = [chet[i] + T[i] for i in range(n // 2)] + \
            [chet[i] - T[i] for i in range(n // 2)]
    return result


def ifft(x):
    n = len(x)
    a = _ifft(x)
    return [a[i] / n for i in range(n)]
    
    
def _ifft(x):
    n = len(x)
    if n == 1: 
        return x
    chet = _ifft(x[0::2])
    nechet =  _ifft(x[1::2])
    T = [npexp(2j * nppi * i / n) * nechet[i] for i in range(n // 2)]
    result = [(chet[i] + T[i]) for i in range(n // 2)] + \
            [(chet[i] - T[i]) for i in range(n // 2)]
    return result

Добавим функцию, которая будет делать нам сетку

In [4]:
def setka(f, n = 1024, a = 0, b = 16):
    step = (b - a) / n
    return [f(a + step * i) for i in range(n)]

Для проверки работоспособности импортируем ещё одну функцию numpy, а так же его коробочное решение

In [5]:
from numpy import allclose 
from numpy.fft import fft as npfft
from numpy.fft import ifft as npifft

Введём какую-нибудь неприятную функцию

In [6]:
def f(x):
    return x**2 + x + (x**3 - x)**0.5

Проверим, что всё работает во все стороны. Для обратного преобразования предлагаю подать ту же сетку, потому что почему бы и нет

In [7]:
allclose(npfft(setka(f)), fft(setka(f)))

True

In [8]:
allclose(npifft(setka(f)), ifft(setka(f)))

True

In [9]:
allclose(npfft(setka(f)), dubolom(setka(f)))

True

In [10]:
allclose(npifft(setka(f)), idubolom(setka(f)))

True

А теперь сравним скорость работы всех этих методов

In [11]:
%%time
a = fft(setka(f))

Wall time: 24 ms


In [12]:
%%time
a = npfft(setka(f))

Wall time: 4 ms


In [13]:
%%time
b = dubolom(setka(f))

Wall time: 1.3 s


Как видим, написанный FFT всего в 6 раз медленнее реализации numpy и в 50 с лищним раз быстрее дуболомного метода